In [1]:
import sys

sys.path.insert(0, 'compiled_protobufs')

In [2]:
""" Generate Taskgraphs from datasets """

from models_datasets.recipe_1m_model import AbstractModelDataset
from models_datasets.recipe_1m_model import Recipe1MModel
from models_datasets.wikihow_model import WikihowModel

dataset_models = {
    "COOKING" : Recipe1MModel,
    "DIY": WikihowModel,
}

for model_name, model in dataset_models.items():
    dataset_model: AbstractModelDataset = model()
    print(f"Dataset Model: {model_name}")
    dataset_model.convert_to_taskgraphs()

Dataset Model: COOKING
Loading taskgraphs...


FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/task-search-quality/task_datasets/recipe1mln_tasks/layer1.json'

In [3]:
import pandas as pd

""" Preprocess and load queries """

# from query_pipeline import query_pipeline
# query_pipeline.preprocess_queries()

queries = {
    "COOKING" : pd.read_csv('queries/cooking.csv'),
    "DIY": pd.read_csv('queries/diy.csv'),
}

queries["COOKING"].head()


,id,raw query,domain,knowledge,complex,many points of view,different key entities,seasonal,regional,target query,short query
0,query-0,how to spatchcock a turkey,cooking,n,y,n,n,y,usa,spatchcock turkey,spatchcock a turkey
1,query-1,I want an easy to make dessert for christmas,cooking,n,y,y,n,y,-,easi dessert christma,easy to make dessert for christmas
2,query-2,any recommendations for a gluten free appetizer,cooking,y,n,y,y,n,-,gluten free appet,gluten free appetizer
3,query-3,how to make a green goddess salad,cooking,y,y,n,y,n,usa,green goddess salad,green goddess salad
4,query-4,how to make a classic english trifle,cooking,y,n,n,y,y,uk,classic english trifl,classic english trifle


In [3]:
queries["DIY"].head()

,id,raw query,domain,knowledge,complexity,many points of view,multiple key entities,subfield,target query,short query
0,query-0,how to choose a good hairstyle for men,diy,y,y,y,n,appearance,choos good hairstyl men,choose a good hirstyle for men
1,query-1,how to learn a new language,diy,n,n,y,n,self improvement,learn new languag,learn a new language
2,query-2,how to remove food stains from a carpet,diy,n,y,n,y,maintanence,remov food stain carpet,remove food stains from a carpet
3,query-3,how to wash a car with less water,diy,n,n,y,y,maintanence,wash car water,wash a car with less water
4,query-4,how do I decorate an office,diy,n,y,y,n,home improvement,decor offic,decorate an office


In [2]:
from models_indexes.bm25_model import BM25Model
from models_indexes.ance_model import AnceModel
from models_indexes.colbert_model import ColbertModel
from models_indexes.marqo_model import MarqoModel
from models_indexes.abstract_model import AbstractModel

models = {}

for domain in  ["COOKING", "DIY"]:
# for domain in  ["COOKING"]:
    models[domain] = {
        # "bm25" : BM25Model(domain = domain),
        # "bm25+rm3" : BM25Model(domain = domain, rm3=True),
        # "bm25+t5" : BM25Model(domain = domain, t5=True),
        # "bm25+rm3+t5" : BM25Model(domain = domain, rm3=True, t5=True),
        "ance+t5": AnceModel(domain = domain, t5=True),
        "colbert+t5": ColbertModel(domain = domain, t5=True),
        # "marqo": MarqoModel(domain = domain),
        # "marqo": MarqoModel(domain = domain),
    }

/home/ubuntu/task-search-quality/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-05 14:06:03.229567: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-05 14:06:04.204029: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-05 14:06:04.204128: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_

In [5]:
from models_indexes.marqo_model import MarqoModel

# model = MarqoModel(domain = "DIY")
# model.build_index()
# model.convert_search_results_to_run(pd, raw=False)
# model.search("")

In [ ]:
import json
path = "indexes/temp/cooking/system_index_marqo/recipe1m+-taskmaps.json"
# path2 = "indexes/temp/cooking/system_index_marqo/recipe1m+-taskmaps copy.json"

with open(path, "r") as f:
    obj = json.load(f)
    
# with open(path2, "r") as f:
#     obj2 = json.load(f)

print(len(obj))
# print(len(obj2))

# obj = obj[995500:]

# with open(path, 'w') as f:
#     f.write(json.dumps(obj, indent=4))

In [ ]:
# build indexes
print("Build indexes")
for domain, index_models in models.items():
    for model_name, model in index_models.items():
        model.build_index()

In [4]:
# create run files
print("Creating run files ")
for domain, index_models in models.items():
    qs = queries[domain]
    print(f"DOMAIN {domain}")
    for model_name, model in index_models.items():
        model.convert_search_results_to_run(qs)
        
        

Creating run files 
DOMAIN COOKING
Run file saved at /home/ubuntu/task-search-quality/measurements/cooking/run_files/ance+t5.run
Run file saved at /home/ubuntu/task-search-quality/measurements/cooking/run_files/TCT-ColBERTv2+t5.run
DOMAIN DIY
Run file saved at /home/ubuntu/task-search-quality/measurements/diy/run_files/ance+t5.run


KeyboardInterrupt: 

In [ ]:
# create empty judgments
print("Creating empty judgments")
for domain, index_models in models.items():
    qs = queries[domain]
    print(f"DOMAIN {domain}")
    for model_name, model in index_models.items():
        model.create_empty_judgments(qs, k=50, n=10)

In [ ]:
from models_indexes.marqo_model import MarqoModel
marqo_diy = MarqoModel(domain = "DIY")
marqo_diy.build_index()
# marqo_diy.search("cleaning ")

In [ ]:
""" Build indexes from datasets """

# from models_indexes.bm25_model import BM25Model

# bm25 = BM25Model(domain = "DIY", t5=True)
# bm25.build_index()

# from models_indexes.ance_model import AnceModel
# ance: AnceModel = AnceModel(domain = "DIY")
# bm25.build_index()

# # bm25_cooking = BM25Model(domain = "DIY", t5=True)
# # bm25_cooking.build_index()

# from models_indexes.colbert_model import ColbertModel
# colbert: ColbertModel = ColbertModel(domain = "DIY")
# colbert.build_index()

# query = queries["COOKING"]["target query"][0]
# query = "pizza"
# colbert_cooking.search(query)
# colbert_diy.convert_search_results_to_run()
# colbert_diy.create_empty_judgments()

# from models_indexes.ance_model import AnceModel
# ance: AnceModel = AnceModel(domain = "COOKING")
# ance.build_index()
# ance.search("easi dessert christma")

# from models_indexes.colbert_model import ColbertModel
# colbert: ColbertModel = ColbertModel(domain = "COOKING")
# colbert.build_index()
# colbert.search("pizza")

# from models_indexes.colbert_model import ColbertModel
# colbert: ColbertModel = ColbertModel(domain = "DIY")
# colbert.build_index()
# colbert.search("kitchen")


In [ ]:
from models_indexes.colbert_model import ColbertModel
colbert: ColbertModel = ColbertModel(domain = "COOKING")
colbert.build_index()
colbert.search("fish")

# marqo_cooking = MarqoModel(domain = "COOKING")
# marqo_cooking.build_index()
# marqo_cooking.search("fish")

# queries_path = os.path.join(os.getcwd(), "queries", "cooking.csv")

# pd_cooking_queries = pd.read_csv(queries_path).iloc[:10]
# bm25_food.convert_search_results_to_run(pd_cooking_queries)
# bm25_food.create_empty_judgments(pd_cooking_queries, 5)
# queries_path = os.path.join(os.getcwd(), "queries", "diy.csv")
# pd_diy_queries = pd.read_csv(queries_path).iloc[:10]
# ance_diy.build_index()
# ance_diy.convert_search_results_to_run(pd_diy_queries)

In [ ]:
# ance_cooking = AnceModel(domain = "COOKING")
# ance_cooking.build_index()

In [ ]:
# from models_indexes.bm25_model import BM25Model
# bm25_cooking = BM25Model(domain = "COOKING")
# bm25_cooking.build_index()

# queries_path = os.path.join(os.getcwd(), "queries", "cooking.csv")
# pd_cooking_queries = pd.read_csv(queries_path).iloc[:10]
# query = "granola"
# bm25_cooking.search(query)